In [873]:
import json
import pennylane as qml
import pennylane.numpy as np

def circuit(oracle):
    """The circuit to find if the Bitland Kingdom was in danger.
    You can query the oracle by calling oracle(wires=[0, 1, 2])."""
    

    # Put your code here #
    # qml.BasisState([1, 1, 0], wires=[0, 1, 2])
    """qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)
    qml.CZ(wires=(0, 1))
    oracle(wires=[0, 1, 2])
    qml.CZ(wires=(2, 0))
    qml.Hadamard(0)
    # qml.GroverOperator(wires=(0, 1, 2))
    qml.CNOT(wires=(0, 2))
    qml.CNOT(wires=(0, 1))
    qml.measure(2, reset=True)
    qml.CNOT(wires=(2, 0))
    qml.Hadamard(wires=0)


    qml.Hadamard(wires=1)
    qml.CNOT(wires=(0, 1))
    qml.Hadamard(wires=2)"""
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 0])
    
    # Apply Oracle to the superposition state
    oracle(wires=[0, 1, 2])






In [883]:
dev = qml.device("default.qubit", wires=3)
oracle, target = oracle_maker()
@qml.qnode(dev)
def test_circuit():
    circuit(oracle)
    return qml.state()

state = test_circuit()
#print(state)
for i, s in enumerate(state):
    print(f"{i:03b}", f"{s:.02}")

[[1, 0], [1, 1], [0, 0], [0, 1]]
000 (-0.71-8.7e-17j)
001 0j
010 0j
011 (-0.71-8.7e-17j)
100 0j
101 0j
110 0j
111 0j


In [718]:
oracle, target = oracle_maker()

[[0, 1], [0, 0], [1, 1], [1, 0]]


In [2]:
def oracle_maker():
    # States order: |0> , |1>, -|0>, -|1>

    indx = [[0, 0], [1, 0], [1, 1], [0, 1]]

    # Village 00 -> |0>
    # Village 10 -> |1>
    # Village 11 -> -|0>
    # Village 01 -> -|1>

    np.random.shuffle(indx)
    print(indx)

    indices_00 = [index for index, value in enumerate(indx) if value == [0, 0]]
    indices_11 = [index for index, value in enumerate(indx) if value == [1, 1]]

    if set([indices_00[0], indices_11[0]]) == set([0, 1]) or set([indices_00[0], indices_11[0]]) == set([2, 3]):
        target = 0
    else:
        target = 1

    def oracle(wires):

        class op(qml.operation.Operation):
            num_wires = 3
            grad_method = None

            def __init__(self, wires, id=None):
                super().__init__(wires=wires, id=id)

            @property
            def num_params(self):
                return 0

            @staticmethod
            def compute_decomposition(wires):
                wires_input = wires[:2]
                wire_output = wires[2]

                ops = []
                ops.append(qml.ctrl(qml.PauliX(wires=wire_output), control=wires_input, control_values=indx[1]))

                ops.append(
                    qml.ctrl(qml.GlobalPhase(np.pi, wires=wire_output), control=wires_input, control_values=indx[2]))

                ops.append(qml.ctrl(qml.PauliX(wires=wire_output), control=wires_input, control_values=indx[3]))
                ops.append(
                    qml.ctrl(qml.GlobalPhase(np.pi, wires=wire_output), control=wires_input, control_values=indx[3]))

                return ops

        return op(wires=wires)

    return oracle, target

In [2]:
# These functions are responsible for testing the solution.

def oracle_maker():
    # States order: |0> , |1>, -|0>, -|1>

    indx = [[0, 0], [1, 0], [1, 1], [0, 1]]

    # Village 00 -> |0>
    # Village 10 -> |1>
    # Village 11 -> -|0>
    # Village 01 -> -|1>

    np.random.shuffle(indx)

    indices_00 = [index for index, value in enumerate(indx) if value == [0, 0]]
    indices_11 = [index for index, value in enumerate(indx) if value == [1, 1]]

    if set([indices_00[0], indices_11[0]]) == set([0, 1]) or set([indices_00[0], indices_11[0]]) == set([2, 3]):
        target = 0
    else:
        target = 1

    def oracle(wires):

        class op(qml.operation.Operation):
            num_wires = 3
            grad_method = None

            def __init__(self, wires, id=None):
                super().__init__(wires=wires, id=id)

            @property
            def num_params(self):
                return 0

            @staticmethod
            def compute_decomposition(wires):
                wires_input = wires[:2]
                wire_output = wires[2]

                ops = []
                ops.append(qml.ctrl(qml.PauliX(wires=wire_output), control=wires_input, control_values=indx[1]))

                ops.append(
                    qml.ctrl(qml.GlobalPhase(np.pi, wires=wire_output), control=wires_input, control_values=indx[2]))

                ops.append(qml.ctrl(qml.PauliX(wires=wire_output), control=wires_input, control_values=indx[3]))
                ops.append(
                    qml.ctrl(qml.GlobalPhase(np.pi, wires=wire_output), control=wires_input, control_values=indx[3]))

                return ops

        return op(wires=wires)

    return oracle, target


def run(case: str) -> str:
    return "No output"


def check(have: str, want: str) -> None:
    for _ in range(100):
        oracle, target = oracle_maker()

        dev = qml.device("default.qubit", wires=3)

        @qml.qnode(dev)
        def test_circuit():
            circuit(oracle)
            return qml.probs(wires=2)

        test_circuit()
        assert [op.name for op in test_circuit.tape.operations].count("op") == 1, "You can use the oracle once."

        assert np.isclose(test_circuit()[1], target), "Wrong answer!"


# These are the public test cases
test_cases = [
    ('No input', 'No output')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input 'No input'...


AssertionError: You can use the oracle once.

In [702]:
import pennylane as qml

dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def distinguish_states():
    #qml.Hadamard(wires=0)  # Apply Hadamard gate to create |+⟩ state
    qml.PauliX(0)
    qml.Hadamard(wires=0)
    return qml.expval(qml.PauliX(0))  # Measure in X-basis (Hadamard basis)

result = distinguish_states()

if result == 1:
    print("The qubit was in the |1⟩ state")
else:
    print("The qubit was in the |+⟩ state")

The qubit was in the |+⟩ state
